In [1]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [2]:
# Coletando os dados:
exp_2022 = pd.read_csv("..\Dados\COMEX\EXP_2022_MUN.csv", delimiter=";")
agropecuaria = pd.read_excel(r"..\Dados\COMEX\sh4_agropecuaria.xlsx")
tabela_final = pd.read_csv(r"..\Dados\Tabela_final\tabela_final.csv")
municipio_escolhidos = pd.read_csv(r"..\Dados\Tabela_final\municipios_potenciais.csv", index_col=0)


# Trantando os dados de exportação:
exp_2022 = exp_2022.merge(agropecuaria, how="inner", left_on="SH4", right_on="Código SH4")
exp_2022.drop(columns=["Código SH4"], inplace=True)

# Agrupamento que retorna a média do KG_Líquido exportado e do Valor de fobização:
exp_2022 = exp_2022.groupby(["CO_MUN", "Denominação nesse estudo"]).agg({"VL_FOB":"mean", "KG_LIQUIDO":"mean"}).reset_index()
exp_2022_sertao = exp_2022[exp_2022["CO_MUN"].isin(tabela_final["IBGE7"])]

# Index para ser os códigos dos municípios:
exp_2022_sertao.set_index("CO_MUN", inplace=True)
municipio_escolhidos.set_index("IBGE7", inplace=True)
tabela_final.set_index("IBGE7", inplace=True)

In [15]:
# Função para calcular a distância em KM, entre duas coordenadas geográficas:
def distancia_circular(df_escolhidos, df_municipios_sertao, threhold):
    dicionario_distancias = {}
    lista_distancias = []
    for municipio_escolhido, row in df_escolhidos[["LATITUDE", "LONGITUDE"]].iterrows():
        for municipio_qualquer, row2 in df_municipios_sertao[["LATITUDE", "LONGITUDE"]].iterrows():
            if municipio_escolhido != municipio_qualquer:
                distancia = geodesic(row.values, row2.values).kilometers
                if distancia <= threhold:
                    lista_distancias.append((municipio_qualquer, distancia))
    
        dicionario_distancias[df_escolhidos.loc[municipio_escolhido, "NOME"]] = lista_distancias
        lista_distancias = []

    return dicionario_distancias



# Função que determina, quais os produtos os municípios vizinhos estão exportando:
def produtos_exportados_vizinhos(dicionario, expor):
    dicionario_exportacao = {}
    lista_exportacao = []
    contagem = 0
    
    for key, value in dicionario.items():
        for municipio_vizinho in value:
            if  expor.isin([municipio_vizinho[0]]).any().any():
                contagem += 1
                produto = expor[expor["CO_MUN"] == municipio_vizinho[0]]["Denominação nesse estudo"].values
                kg_liq = expor[expor["CO_MUN"] == municipio_vizinho[0]]["KG_LIQUIDO"].values
                valor_fb = expor[expor["CO_MUN"] == municipio_vizinho[0]]["VL_FOB"].values
                
                lista_exportacao.append((municipio_vizinho[0], produto, kg_liq, valor_fb))
                display(expor[expor["CO_MUN"] == municipio_vizinho[0]])

        if contagem:
            print(f"{key}\n")
            dicionario_exportacao[key] = lista_exportacao
        contagem = 0
        lista_exportacao = []
        
    return dicionario_exportacao

### Hipótese 1: Nos municípios selecionados, há exportação de algum tipo de produto.

In [3]:
expo_municipios_escolhidos = municipio_escolhidos.merge(exp_2022_sertao, left_index=True, right_index=True, how="inner")
expo_municipios_escolhidos

,UF,NOME,LATITUDE,LONGITUDE,DIST_EF_PUB,DIST_EM_PUB,IDEB_AI,IDEB_AF,DOCSUP_EF_PUB,DOCSUP_EM_PUB,...,T_ANALF25M,QUAL_MED_AGUA,DIST_CORPO_AGUA,TRANSPORT_COST,TOT_ESC_POR_POP,TOT_MAT_POR_POP,CLUSTER,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
IBGE7,,,,,,,,,,,,,,,,,,,,,


Conclusão:

- Não há exportação de nenhum produto nos municípios escolhidos, bseado nos dados de exportação de 2022.

### Hipótese 2: Nos municípios vizinhos (50km), há exportação de algum tipo de produto.

In [16]:
# Rodando as funções:
distancias_municipios_escolhidos = distancia_circular(municipio_escolhidos, tabela_final, 50)
dicionario_export = produtos_exportados_vizinhos(distancias_municipios_escolhidos, exp_2022)

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
687,2922904,Sumos de frutas (incluídos os mostos de uvas) ...,248108.545455,82893.25


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
610,2807402,Coprodutos do Algodão,22545.0,1526.0


ITAPICURU



,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
530,2606002,Coprodutos do milho,29500.8,33765.2
531,2606002,Farelo/Farinha e Resíduos Sólidos,29500.8,33765.2


CAETÉS



...

,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
504,2504009,Papel,1840.5,416.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
505,2506301,Carne de Frango e Miudezas,34305.44,57228.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
510,2512408,Papel,1351.0,109.0


,CO_MUN,Denominação nesse estudo,VL_FOB,KG_LIQUIDO
513,2515104,Algodão,60623.0,11565.0


ALAGOA GRANDE

